In [ ]:
import os
from osgeo import gdal

# Définir les chemins des fichiers et des dossiers
band_path = "../../data/images"
band_numbers = ['2', '3', '4', '5', '6', '7', '8', '8A', '11', '12']
band_date = [
    "_20220326-105856-076_L2A_T31TCJ_C_V3-0_FRE_B",
    "_20220405-105855-542_L2A_T31TCJ_C_V3-0_FRE_B",
    "_20220714-105903-040_L2A_T31TCJ_C_V3-0_FRE_B",
    "_20220922-105859-863_L2A_T31TCJ_C_V3-1_FRE_B",
    "_20221111-105858-090_L2A_T31TCJ_C_V3-1_FRE_B",
    "_20230219-105857-687_L2A_T31TCJ_C_V3-1_FRE_B"
]  # Liste des dates pour chaque image

# Générer les chemins complets pour chaque bande
band_files = [f"{band_path}/SENTINEL2B{date}{band_number}.tif" for date, band_number in zip(band_date, band_numbers)]

# Répertoire de sortie
out_path = "../results/data/img_pretraitees"
mask_file = os.path.join(out_path, "masque_foret.tif")  # Fichier de masque

# Paramètres de sortie
output_file = os.path.join(out_path, "Serie_temp_S2_allbands.tif")
projection = "EPSG:2154"
pixel_size = 10  # Résolution spatiale en mètres
nodata_value = 0

# Fonction pour reprojeter et rééchantillonner une bande
def reproject_and_resample(band_file, output_file, pixel_size, projection):
    # Ouvrir le fichier de la bande comme un dataset GDAL
    band_ds = gdal.Open(band_file)
    if band_ds is None:
        raise Exception(f"Erreur lors de l'ouverture du fichier {band_file}")

    # Reprojection et rééchantillonnage
    gdal.Warp(
        output_file,
        band_ds,
        xRes=pixel_size,
        yRes=pixel_size,
        dstSRS=projection,
        dstNodata=nodata_value
    )
    band_ds = None  # Fermer le dataset après l'opération

# Étape 1 : Reprojeter et rééchantillonner toutes les bandes
reprojected_bands = []
for i, band_file in enumerate(band_files):
    reprojected_file = os.path.join(out_path, f"date_{i+1}_reprojected.tif")
    reproject_and_resample(band_file, reprojected_file, pixel_size, projection)
    reprojected_bands.append(reprojected_file)

# Étape 2 : Reprojeter et rééchantillonner le masque forêt pour correspondre aux dimensions des bandes
def reproject_and_resample_mask(mask_file, output_file, pixel_size, projection):
    mask_ds = gdal.Open(mask_file)
    if mask_ds is None:
        raise Exception(f"Erreur lors de l'ouverture du fichier {mask_file}")
    
    # Reprojection et rééchantillonnage du masque
    gdal.Warp(
        output_file,
        mask_ds,
        xRes=pixel_size,
        yRes=pixel_size,
        dstSRS=projection,
        dstNodata=nodata_value
    )
    mask_ds = None  # Fermer le dataset après l'opération

# Reprojeter et rééchantillonner le masque
reprojected_mask_file = os.path.join(out_path, "masque_foret_reprojected.tif")
reproject_and_resample_mask(mask_file, reprojected_mask_file, pixel_size, projection)
mask_file = reprojected_mask_file  # Remplacer par le masque reprojeté

# Étape 3 : Appliquer le masque forêt à chaque bande
masked_bands = []
for i, band_file in enumerate(reprojected_bands):
    masked_file = os.path.join(out_path, f"band_{i+1}_masked.tif")
    gdal_calc_command = (
        f"gdal_calc.py -A {band_file} -B {mask_file} --outfile={masked_file} "
        f"--calc=\"A*B\" --NoDataValue={nodata_value} --quiet"
    )
    os.system(gdal_calc_command)
    masked_bands.append(masked_file)

# Étape 4 : Fusionner toutes les bandes dans un seul fichier
# Utilisation de gdal_merge.py pour fusionner les bandes
merge_command = (
    f"gdal_merge.py -separate -o {output_file} " + " ".join(masked_bands) + f" -a_nodata {nodata_value}"
)
os.system(merge_command)

# Vérifier la taille du fichier et convertir en uint8 si nécessaire
output_uint8_file = output_file.replace(".tif", "_uint8.tif")
if os.path.getsize(output_file) > 10 * 1024 * 1024 * 1024:  # Si le fichier est plus grand que 10 Go
    dynamic_convert_command = (
        f"otbcli_DynamicConvert -in {output_file} -out {output_uint8_file} uint8"
    )
    os.system(dynamic_convert_command)
    print(f"Fichier converti en uint8 : {output_uint8_file}")
else:
    print(f"Fichier final généré : {output_file}")

# Supprimer les fichiers intermédiaires
for band in reprojected_bands:
    remove_file(band)
for band in masked_bands:
    remove_file(band)
remove_file(reprojected_mask_file)  # Supprimer le masque reprojeté


0...10...20...30...40...50...60...70...80...90...100 - done.
Fichier final généré : ../results/data/img_pretraitees/Serie_temp_S2_allbands.tif
